# Custome Loss optimization 
in this notebook we will look at using custom loss in keras to optimize mean revenue for discount offer for leaving customers
## introduction
our goal is to offer the highest price that the user will accsept without exeding his/hers spend limit
its like playing "The price is right" if we go over the spend limit or revune is zero

also to be cost efective we canot offer less then 55 dolar for our service
in this notebook we will
* load and fix the data to our use case (Anima streming service)
* preprocessing 
* create an evalution metric
* define our metric of revune and evaluate baseline model (mean spend limit$ sale offer for all)
* create a base NN to predict spend limit
* creat a NN with custome loss and output


# load and fix the data to our use case

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.figure_factory as ff
import os
import seaborn
from sklearn.model_selection import train_test_split
import tensorflow as tf

data = pd.read_csv("/kaggle/input/churn-modeling-dataset/Churn_Modelling.csv").drop(
    ["RowNumber", "CustomerId", "Surname"], axis=1
)
data.head()

In [ ]:
# lets make the data more anima like
data.columns = [
    "SpendLimit",
    "Geography",
    "Gender",
    "Age",
    "Tenure",
    "TotalWatchTime",
    "VODRental",
    "HasCrCard",
    "IsActiveMember",
    "LastMonthWatchTime",
    "PreferDub",
]
data["SpendLimit"] = data["SpendLimit"] / 10
data.head(3).T

# create an evalution metric

In [ ]:

#lets visulize our profit from a user with a spending limit of 62$
x = np.asarray(range(100) )
y =np.asarray(range(100) )
y[y<5]=0
y[y>62]=0
pd.DataFrame([x,y]).T.set_index(0).plot()

In [ ]:
y_true_sample = np.asarray([30,40,50])
y_pred_sample = np.asarray([20,50,40])

In [ ]:
def mean_profit(y_true,y_pred):
    return ((y_pred<y_true).astype(int) *y_pred).mean()
mean_profit(y_true_sample,y_pred_sample)
    

# preprocessing

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

enc = OneHotEncoder(handle_unknown="ignore")
stander_scaler = StandardScaler()
label_encoder = LabelEncoder()

X = np.concatenate(
    (
        ## OneHotEncoder
        enc.fit_transform(data[["Geography"]]).toarray(),
        ## Stander Scaler
        stander_scaler.fit_transform(
            data[
                [
                    "Age",
                    "Tenure",
                    "TotalWatchTime",
                    "VODRental",
                    "LastMonthWatchTime",
                ]
            ]
        ),
        ## LabelEncoder
        label_encoder.fit_transform(data[["Gender"]]).reshape(-1, 1),
        ## No formatation
        data[["HasCrCard", "IsActiveMember"]].values,
    ),
    axis=1,
)

y = data["SpendLimit"].values
X.shape,y.shape

# Train ANN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D
import keras


def get_model():
    return Sequential(
        [
            Dense(units=200, input_dim=11, activation="relu"),
            Dense(150, activation="relu"),
            Dropout(0.2),
            Dense(100, activation="relu"),
            Dense(100, activation="relu"),
            Dropout(0.2),
            Dense(100, activation="relu"),
            Dense(100, activation="relu"),
            Dense(100, activation="relu"),
            Dropout(0.2),
            Dense(100, activation="relu"),
            Dense(1, activation="relu"),
        ]
    )


def train_ann(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = get_model()

    model.compile(
        optimizer="adam", loss="mae", metrics=["mse"],
    )

    # Trainig and returning back the results.
    history = model.fit(
        X_train,
        y_train,
        batch_size=100,
        epochs=5,
        verbose=1,
        validation_data=(X_test, y_test),
    )
    y_pred = model.predict(X_test)
    score = mean_profit(y_test,y_pred)
    print (f"your score is :{score} with min pred of {y_pred.min()} ")

In [ ]:
train_ann(X, y)

# Optimize



In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D,Lambda,BatchNormalization
import keras
import keras.backend as K


def get_model2():
    return Sequential(
        [
            Dense(units=200, input_dim=11, activation="relu"),
            Dense(150, activation="relu"),
            Dropout(0.2),
            Dense(100, activation="relu"),
            Dense(100, activation="relu"),
            Dropout(0.2),
            Dense(100, activation="relu"),
            Dense(100, activation="relu"),
            Dense(100, activation="relu"),
            Dropout(0.2),
            Dense(100, activation="relu"),
            Dense(1, activation="relu"),
            #Lambda(lambda x: K.clip(x,55,1000))
        ]
    )
def customLoss(y_true,y_pred):
    return -K.mean(K.cast(y_pred<y_true,"float32") *y_pred - K.cast(y_pred>y_true,"float32") *(y_pred- y_true))
def customLoss_naive(y_true,y_pred):
    return -K.mean((K.cast(y_pred<y_true,"float32") *y_pred))

def train_ann_custom(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = get_model2()

    model.compile(
        optimizer="adam", loss=customLoss, metrics=["mse"],
    )

    # Trainig and returning back the results.
    history = model.fit(
        X_train,
        y_train,
        batch_size=50,
        epochs=50,
        verbose=1,
        validation_data=(X_test, y_test),
    )
    y_pred = model.predict(X_test)
    score = mean_profit(y_test,y_pred)
    print (f"your score is :{score} with min pred of {y_pred.min()} ")
train_ann_custom(X, y)